If you run transformers, there is one regex that seems to pick up an extra line with some comment that has 'def' in it - fix it or just use the files in the data folder that are manually patched to remove that extra line

In [1]:
from tensorflow.python.client import device_lib
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, GRU, Input, TimeDistributed, Dropout
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

In [2]:
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13188000061781736241
xla_global_id: -1
]


2022-07-29 23:55:04.186593: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
bas_input = open("./data/input_bas.txt", 'r').readlines()
py_input = open("./data/input_py.txt", 'r').readlines()

In [5]:
len(bas_input) == len(py_input)

True

In [6]:
    tokenizer_bas = Tokenizer(filters=[])
    tokenizer_bas.fit_on_texts(bas_input)
    seq_bas = tokenizer_bas.texts_to_sequences(bas_input)
    seq_bas = pad_sequences(seq_bas, padding='post')

    tokenizer_py = Tokenizer(filters=[])
    tokenizer_py.fit_on_texts(py_input)
    seq_py = tokenizer_py.texts_to_sequences(py_input)
    seq_py = pad_sequences(seq_py, padding='post')

    seq_py = seq_py.reshape(*seq_py.shape, 1)

In [7]:
tmp_seq_bas = pad_sequences(seq_bas, maxlen=seq_py.shape[1], padding='post')
tmp_seq_bas = tmp_seq_bas.reshape((-1, seq_py.shape[-2], 1))

In [ ]:
py_input

In [107]:
m = 0
for l in py_input:
    if len(l) > m:
        m = len(l)
m

11911

In [103]:
seq_py.shape

(3141, 1118)

In [ ]:
list(seq_bas[0])

In [8]:
model1 = Sequential()
model1.add(GRU(64, input_shape=tmp_seq_bas.shape[1:], return_sequences=True)) #256
model1.add(TimeDistributed(Dense(256, activation='relu')))#1024
model1.add(Dropout(0.5))
model1.add(TimeDistributed(Dense(len(tokenizer_py.word_index), activation='softmax'))) 

model1.compile(loss=sparse_categorical_crossentropy, optimizer=Adam(0.05), metrics=['accuracy'])

In [ ]:
model1.fit(tmp_seq_bas, seq_py, batch_size=1024, epochs=10, validation_split=0.2)

Epoch 1/10


In [1]:
import collections

In [10]:
x = [1,2,3,4,5,5,6,7,9,0,0,12,13,14]
[q for q,k in collections.Counter(x).items() if k > 1]

[5, 0]

In [7]:
collections.Counter(x).items()

dict_items([(1, 1), (2, 1), (3, 1), (4, 1), (5, 2), (6, 1), (7, 1), (9, 1), (0, 2), (12, 1), (13, 1), (14, 1)])